In [1]:
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys
sys.path.append('../code')
import d2lzh_pytorch as d2l
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
(corpus_indices,char_to_idx,idx_to_char,vocab_size)=d2l.load_data_jay_lyrics()

In [3]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
print('will use',device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)
    return nn.ParameterList([W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh, W_hh,b_h,W_hq,b_q])

will use cuda


In [4]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [5]:
def gru(inputs,state,params):
    W_xz,W_hz,b_z,W_xr, W_hr, b_r,W_xh, W_hh, b_h,W_hq,b_q=params
    H,=state
    outputs=[]
    for X in inputs:
        Z=torch.sigmoid(torch.matmul(X,W_xz)+torch.matmul(H,W_hz)+b_z)
        R=torch.sigmoid(torch.matmul(X,W_xr)+torch.matmul(H,W_hr)+b_r)
        H_tilda=torch.tanh(torch.matmul(X,W_xh)+torch.matmul(R*H,W_hh)+b_h)
        H=Z*H+(1-Z)*H_tilda
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

In [6]:
num_epochs,num_steps,batch_size,lr,clipping_theta=160,35,32,1e2,1e-2
pred_period,pred_len,prefixes=40,50,['分开','不分开']

In [7]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 150.670404, time 0.30 sec
 - 分开 我想你的让我 你不你 我想你 你不你 我想你 你不你 我想你 你不你 我想你 你不你 我想你 你不
 - 不分开 我想你的让我 你不你 我想你 你不你 我想你 你不你 我想你 你不你 我想你 你不你 我想你 你不
epoch 80, perplexity 33.362577, time 0.30 sec
 - 分开 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
epoch 120, perplexity 6.016949, time 0.30 sec
 - 分开 我想就这样牵着你 想想和你的微色 我想想你的微笑每天都能看到  我知道这里很美但  说  说去了考
 - 不分开 我知不这样 我知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该
epoch 160, perplexity 1.785895, time 0.30 sec
 - 分开 让我 没你了骑色着我 想散 你想很久了吧? 我的认真败给黑色幽默 让我开你以相 不知不觉 你已经离
 - 不分开 如果我怕已经是一个悲剧 我想我这辈子 但这依依不舍  隔壁让回都都有一直 融到让心宙里 什么都每 


In [8]:
lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)


epoch 40, perplexity 1.020230, time 0.16 sec
 - 分开始想像 爸和妈当年的模样 说著一口吴侬软语的姑娘缓缓走过外滩 消失的 旧时光 一九四三 回头看 的片
 - 不分开始共渡每一天 手牵手 一步两步三步四步望著天 看星星 一颗两颗三颗四颗 连成线背著背默默许下心愿 看
epoch 80, perplexity 1.012335, time 0.16 sec
 - 分开始移动 一个马步向前 一记左钩拳 右钩拳 一句惹毛我的人有危险 一再重演 一根我不抽的菸 一放好多年
 - 不分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
epoch 120, perplexity 1.020221, time 0.16 sec
 - 分开的话像语言暴力 我已无能为力再提起 决定中断熟悉 然后在这里 不限日期 然后将过去 慢慢温习 让我爱
 - 不分开 干什么 干什么 日行千里系沙袋 飞檐走壁莫奇怪 去去就来 一个马步向前 一记左钩拳 右钩拳 一句惹
epoch 160, perplexity 1.215967, time 0.16 sec
 - 分开不投 整颗心到现在还在抽痛 还为分手前那句抱歉 在感动 穿梭时间的画面的钟 从反方向开始移动  你的
 - 不分开不投 又不会掩护我 选你这种队友 瞎透了我 说你说 分数怎么停留 一直在停留 谁让它停留的溪边河口 
